##### Flow:
```text
User Question
    ↓  
Embed question  
    ↓  
Vector similarity search
    ↓  
Top-K chunks
    ↓  
Prompt assembly
    ↓  
LLM call
    ↓  
Final answer
```

##### Prequisite
You will have two Azure OpenAI deployments:

Create in Azure AI Foundry / Azure OpenAI:
- Embedding deployment (example name): text-embedding-3-small
- Chat deployment (example name): gpt-4o-mini

Azure OpenAI API calls use deployment names (not the underlying model name).

##### Input and Output:
- Input:
    - Table: `databricks_rag_demo.default.azure_compute_doc_embeddings`
    - User question

- Output: User's answer

Important design choice: How to do vector search?

We have 3 options:

- Option A — Naive cosine similarity in Spark (we'll start here)
	- Simple
	- Transparent
	- Works for thousands of chunks
	- Good for learning

- Option B — Databricks Vector Search
	- Production-grade
	- Scalable
	- Index-based
	- We can move here later

- Option C — External vector DB (Pinecone, Weaviate, etc.)
	- Overkill right now

We'll start with Option A, then upgrade.

#### 04 - Retrieval-Augmented Generation (RAG)

This notebook implements a basic RAG pipeline:
1. Embed user queries
2. Retrieve relevant document chunks
3. Assemble a grounded prompt
4. Generate an answer using an LLM

Vector Search (production retrieval)

1. Create the Vector Search endpoint + index (recommended via UI)

- Databricks → Compute -> Vector Search
- Create endpoint (name example): vs_azure_compute
- Create index
  - Type: Delta Sync index
  - Name: azure_compute_docs_vs_index
  - Source table: databricks_rag_demo.default.azure_compute_doc_embeddings
  - Primary key: chunk_id
	- Embedding column: embedding

2. Query the index in Python

Once endpoint + index exist, use the Vector Search client.

In [0]:
%run ./00_install_deps_and_restart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/948.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 809.0/948.6 kB 25.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/113.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/361.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [0]:
%run ./00_constants

In [0]:
%run ./00_utils

In [0]:
%run ./00_init_openai_client

In [0]:
import mlflow
# Disable mlflow autologging
mlflow.autolog(disable=True)
mlflow.openai.autolog(disable=True)


Vector Search (production retrieval)

3A) Create the Vector Search endpoint + index (recommended via UI)

Fastest/least pain is UI:
- Databricks -> Compute -> Vector Search
- Create endpoint (name example): azure_compute_docs
- Create index
  - Type: Delta Sync index
  - Source table: databricks_rag_demo.default.azure_compute_docs_embeddings
  - Primary key: chunk_id
	- Embedding column: embedding

3B) Query the index in Python

Once endpoint + index exist, use the Vector Search client.


In [0]:
questions = [
    "What is the difference between a normal Azure VM and an ephemeral VM?",
    "How do I resize an Azure virtual machine?",
    "What is Azure VM Scale Sets?",
    # "How does Azure handle VM disk persistence?",
    # "What is the difference between Spot VM and normal VM?",
    # "How do I enable accelerated networking?",
    # "What is Azure availability set vs availability zone?"
]

batch_ask(questions, k=5, retrievers=["A", "B"], do_eval=True, verbose=False)


Question: What is the difference between a normal Azure VM and an ephemeral VM?

--- Retriever A ---

--- Retriever B ---
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.

Question: How do I resize an Azure virtual machine?

--- Retriever A ---

--- Retriever B ---
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentic

[{'query_id': 'e9f6e839-c5ad-415b-9cf0-6ad44ecc2a0f',
  'question': 'What is the difference between a normal Azure VM and an ephemeral VM?',
  'answer': 'The primary difference between a normal Azure VM and an ephemeral VM lies in the storage configuration of the operating system disk.\n\n1. **Normal Azure VM**:\n   - A normal Azure VM typically uses a standard managed OS disk that is persistent. This means that the OS disk retains its data even when the VM is stopped or deallocated. The OS disk can be backed up and restored, and it is suitable for applications that require data persistence.\n\n2. **Ephemeral VM**:\n   - An ephemeral VM, on the other hand, uses an ephemeral OS disk. This type of disk is temporary and does not retain data once the VM is stopped or deallocated. The data on an ephemeral OS disk is lost when the VM is deallocated, making it suitable for stateless applications or workloads that can tolerate interruptions. Ephemeral disks are typically faster because they ar